In [ ]:
import os
import shutil

# Create directories for training and testing sets
os.mkdir('train')
os.mkdir('test')
# Copy the first 10,000 cat images to the training set
cat_files = os.listdir('cats_and_dogs/train/cats')
for file_name in cat_files[:10000]:
    shutil.copy(f'cats_and_dogs/train/cats/{file_name}', 'train')
# Copy the first 10,000 dog images to the training set
dog_files = os.listdir('cats_and_dogs/train/dogs')
for file_name in dog_files[:10000]:
    shutil.copy(f'cats_and_dogs/train/dogs/{file_name}', 'train')
# Copy the next 2,500 cat images to the testing set
for file_name in cat_files[10000:12500]:
    shutil.copy(f'cats_and_dogs/train/cats/{file_name}', 'test')
# Copy the next 2,500 dog images to the testing set
for file_name in dog_files[10000:12500]:
    shutil.copy(f'cats_and_dogs/train/dogs/{file_name}', 'test')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=500)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.save('dog_cat_classifier.h5')